In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱한다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러온다.
import pandas as pd

from tqdm import trange

In [2]:
# 크롤링 할 사이트
base_url = "https://finance.naver.com/news/news_list.nhn?mode=RANK&page=2"
response = requests.get( base_url )
response

<Response [200]>

In [3]:
soup = bs(response.text, 'html.parser')

content = soup.select("div.hotNewsList")
list_content = str(content).split("\n")

In [ ]:
list_result = []

for page in range(10):
    base_url = f"https://finance.naver.com/news/news_list.nhn?mode=RANK&page={page+1}"
    response = requests.get( base_url )
    response
    
    soup = bs(response.text, 'html.parser')

    content = soup.select("div.hotNewsList")
    list_content = str(content).split("\n")
    
    for str_content in list_content:
        if "href" in str_content:
            try:
                head_line = str_content.split('title=')[1]
            except:
                head_line = str_content

            list_result.append(head_line[1:].replace("&quot;","").replace("</a>","").replace("&amp;","&").split('">')[0])
            
list_result = list(set(list_result))

list_result

In [21]:
import pandas as pd
jongmok_list_csv_file = 'jongmok_list.csv'

# 종목 명 딕셔너리 생성
dict_jongmok_nm = {}

# 종목 딕셔너리 생성
def make_jongmok_dict():
    # CSV 데이터 DataFrame으로 추출
    df_jongmok = pd.read_csv(jongmok_list_csv_file, encoding="CP949")
    # DataFrame에서 딕셔너리 생성
    try:
        df_jongmok = df_jongmok[["단축코드", "한글 종목약명"]]
        for index, row in df_jongmok.iterrows():
            dict_jongmok_nm["A" + row["단축코드"]] = row["한글 종목약명"]
    except:
        pass
    
make_jongmok_dict()

dict_jongmok_nm["A000660"]

'SK하이닉스'

In [ ]:
def match_full_name(row, nm):
    row = row.replace('"','').replace(","," ").replace("…"," ").replace("...", " ").replace("·"," ")
    list_row = row.split(" ")
    for val in list_row:
        if nm == val:
            return True
    return False

list_whole = []
for cd, nm in dict_jongmok_nm.items():
    for row in list_result:
        list_select = []
        if nm in row:
            if match_full_name(row, nm):
                list_select.append(cd)
                list_select.append(nm)
                list_select.append(row.replace("]","").replace("[",""))
                list_whole.append(list_select)
                
list_cols = ["종목코드", "종목명", "기사"]
df_news = pd.DataFrame(list_whole, columns=list_cols)
df_news = df_news.sort_values(by=["종목코드"])
df_news
# for list_row in list_whole:
#     print(list_row)

In [ ]:
# 종목의 마감정보 추출
list_jongmok_code = ['122630', '005930','352820']
list_cols = ["전일", "고가", "거래량", "시가", "저가", "거래대금"]
list_whole = []
for jongmok_code in list_jongmok_code:
    base_url = "https://finance.naver.com/item/main.nhn?code=" + jongmok_code
    response = requests.get( base_url, headers={"User-agent": "Mozilla/5.0"} )
    soup = bs(response.text, 'html.parser')
    idx = 0
    list_jongmok = []
    for href in soup.find("div",{"class":"rate_info"}).find_all("td"):
        list_href = str(href).split("\n")
        idx += 1
        key = list_href[1].replace("</span>", "").split(">")[1]
        val = list_href[3].split("</span>")[0].split(">")[1].replace(",", "")
        #print(jongmok_code, key, val)
        list_jongmok.append("20210414")
        list_jongmok.append(jongmok_code)
        list_jongmok.append(val)
        list_whole.append(list_jongmok)
list_whole   

In [34]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

def adjust_title(headline):
    result_string = headline.split("title=")[1].split("</a>")[0]
    result_string = result_string.split('">')[0]
    return result_string

def adjust_wdate(headline):
    result_string = "20" + headline.split(">")[1].split(" ")[0]
    result_string = result_string.replace(".","-")
    return result_string

def match_full_name(row, nm):
    row = row.replace('"','').replace(","," ").replace("…"," ").replace("...", " ").replace("·"," ")
    list_row = row.split(" ")
    for val in list_row:
        if nm == val:
            return True
    return False

list_feature = []
for page in range(10):
    base_url = f"https://finance.naver.com/news/market_special.nhn?&page={page+1}"
    response = requests.get( base_url, headers={"User-agent": "Mozilla/5.0"} )
    soup = bs(response.text, 'html.parser')
    end_tf = False
    
    list_line = []
    for row in soup.find("div",{"class":"boardList2"}).find_all("td"):
        if end_tf: break
        list_row = str(row).split("\n")
        for headline in list_row:
            if "title" in headline:
                list_line.append(adjust_title(headline))
            elif "wdate" in headline:
                wdate = adjust_wdate(headline)
                if wdate < "2021-06-24":
                    end_tf = True
                    break
                list_line.append(wdate)
                list_feature.append(list_line)
                list_line = []
            else:
                continue
                
for article in list_feature:
    for cd, nm in dict_jongmok_nm.items():
        if match_full_name(article[0], nm):
            print(cd, nm, article)

A139480 이마트 ['"이베이코리아 인수 이마트 5% 급등(종합2보)', '2021-06-25']
A950130 엑세스바이오 ['"엑세스바이오, 코로나19 항체 진단키트 FDA 승인...15%대 강세', '2021-06-25']
A151910 나노스 ['"나노스, 모더나 사용 LNP 생산 계약 소식에 급등', '2021-06-25']
A047810 한국항공우주 ['"한국항공우주, 공공기관 입찰 자격 1년 6개월 정지...주가 약세', '2021-06-25']
A084180 수성이노베이션 ['"\'이재명 관련주\' 수성이노베이션 강세', '2021-06-25']
A139480 이마트 ['"이베이코리아 인수 이마트 강세(종합)', '2021-06-25']
A000720 현대건설 ['"현대건설, 실적 개선 기대감에 ‘강세’', '2021-06-25']
A203690 프로스테믹스 ['"프로스테믹스, 윤석열 대권선언에 서울중앙지검 출신 감사 부각 강세', '2021-06-25']
A203690 프로스테믹스 ['"프로스테믹스, 코로나19 치료 \'악템라\' 효과증명...美FDA 긴급사용승인 획득↑', '2021-06-25']
A139480 이마트 ['"이베이코리아 인수 이마트 3%대 상승', '2021-06-25']
A067160 아프리카TV ['"아프리카TV, 52주 신고가…3거래일 연속 상승', '2021-06-25']
A122450 KMH ['"KMH, 546억 규모 자사주 취득 결정에 강세', '2021-06-25']
A192080 더블유게임즈 ['"더블유게임즈, 자회사 나스닥 상장 연기 소식에 \'약세\'', '2021-06-25']
A035720 카카오 ['"카카오, 차익 매물 출회에 2거래일 연속 ‘약세’', '2021-06-25']
A204630 스튜디오산타클로스 ['"스튜디오산타클로스, 608억원 규모 유증 추진에 \'약세\'', '2021-06-25']
A120110 코오롱인더 ['"코오롱인더, 52주 신고가…2분기 호실적 기대', '2021-06-